# Stock NeurIPS2018 Part 3. Backtest
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the third and last part of the NeurIPS2018 series, introducing how to use use the agents we trained to do backtest, and compare with baselines such as Mean Variance Optimization and DJIA index.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [102]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [103]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

from rl_traders.finrl.finrl.agents.stablebaselines3.models import DRLAgent
from rl_traders.finrl.finrl.config import INDICATORS, TRAINED_MODEL_DIR
from rl_traders.finrl.finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from rl_traders.finrl.finrl.meta.preprocessor.yahoodownloader import YahooDownloader

%matplotlib inline

# Part 2. Backtesting

To backtest the agents, upload trade_data.csv in the same directory of this notebook. For Colab users, just upload trade_data.csv to the default directory.

In [104]:
train = pd.read_csv('train_data.csv')
trade = pd.read_csv('trade_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

Then, upload the trained agent to the same directory, and set the corresponding variable to True.

In [105]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

Load the agents

In [106]:
trained_a2c = A2C.load(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ddpg = DDPG.load(TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None
trained_ppo = PPO.load(TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None
trained_td3 = TD3.load(TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None
trained_sac = SAC.load(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

e:\Development\Anaconda3\envs\trading\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
e:\Development\Anaconda3\envs\trading\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export 

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [107]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [108]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [109]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [110]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym) if if_using_a2c else (None, None)

hit end!


In [111]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym) if if_using_ddpg else (None, None)

hit end!


In [112]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym) if if_using_ppo else (None, None)

hit end!


In [113]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

hit end!


In [114]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym) if if_using_sac else (None, None)

hit end!


# Part 3: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [115]:
def process_df_for_mvo(df):
  return df.pivot(index="date", columns="tic", values="close")

### Helper functions for mean returns and variance-covariance matrix

In [116]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai

# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns): 
  import numpy as np 
  StockReturn = np.zeros([Rows-1, Columns]) 
  for j in range(Columns):        # j: Assets 
    for i in range(Rows-1):     # i: Daily Prices 
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100 
      
  return StockReturn

### Calculate the weights for mean-variance

In [117]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

array([[242.123],
       [229.81 ],
       [224.915],
       [221.147],
       [221.481],
       [229.81 ],
       [224.238],
       [233.038],
       [233.99 ],
       [232.008],
       [231.684],
       [237.541],
       [242.574],
       [245.92 ],
       [226.946],
       [222.462],
       [231.409],
       [227.74 ],
       [223.855],
       [215.938],
       [210.198],
       [217.203],
       [223.561],
       [224.542],
       [220.264],
       [238.385],
       [242.437],
       [236.982],
       [237.394],
       [237.827],
       [237.778],
       [237.325],
       [238.142],
       [241.074],
       [243.583],
       [243.494],
       [237.856],
       [236.45 ],
       [251.02 ],
       [250.578],
       [250.902],
       [246.16 ],
       [241.162],
       [240.424],
       [243.405],
       [241.457],
       [248.433],
       [252.772],
       [253.067],
       [244.989],
       [240.739],
       [236.568],
       [237.896],
       [240.483],
       [234.344],
       [23

In [118]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
 
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.105]
Variance-Covariance matrix of returns
 4.341364379077664


### Use PyPortfolioOpt

In [119]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(len(cleaned_weights_mean))])
mvo_weights

ValueError: Covariance matrix does not match expected returns

In [ ]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

array([3326.984,    0.   ,    0.   ,    0.   ,  145.392,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,  979.214,    0.   ,    0.   ,    0.   ,    0.   ])

In [ ]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
MVO_result

,Mean Var
date,
2022-10-06,9.938709e+05
2022-10-07,9.626415e+05
2022-10-10,9.582084e+05
2022-10-11,9.545301e+05
2022-10-12,9.525360e+05
...,...
2023-03-30,1.014359e+06
2023-03-31,1.025741e+06
2023-04-03,1.050658e+06


# Part 4: S&P500 index

Add S&P500 index as a baseline to compare with.

In [ ]:
TRAIN_START_DATE = '2019-10-05'
TRAIN_END_DATE = '2022-10-05'
TRADE_START_DATE = '2022-10-06'
TRADE_END_DATE = '2023-04-10'

In [ ]:
# sp_500 = YahooDownloader(
#     start_date=TRADE_START_DATE, end_date=TRADE_END_DATE, ticker_list=["^SPX"]
# ).fetch_data()

In [ ]:
# sp_500 = sp_500[["date", "close"]]
# fst_day = sp_500["close"][0]
# dji = pd.merge(
#     sp_500["date"],
#     sp_500["close"].div(fst_day).mul(1000000),
#     how="outer",
#     left_index=True,
#     right_index=True,
# ).set_index("date")

<a id='4'></a>
# Part 5: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
df_result_a2c = (
    df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
    if if_using_a2c
    else None
)
df_result_ddpg = (
    df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
    if if_using_ddpg
    else None
)
df_result_ppo = (
    df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
    if if_using_ppo
    else None
)
df_result_td3 = (
    df_account_value_td3.set_index(df_account_value_td3.columns[0])
    if if_using_td3
    else None
)
df_result_sac = (
    df_account_value_sac.set_index(df_account_value_sac.columns[0])
    if if_using_sac
    else None
)

result = pd.DataFrame(
    {
        "a2c": df_result_a2c["account_value"] if if_using_a2c else None,
        "ddpg": df_result_ddpg["account_value"] if if_using_ddpg else None,
        "ppo": df_result_ppo["account_value"] if if_using_ppo else None,
        "td3": df_result_td3["account_value"] if if_using_td3 else None,
        "sac": df_result_sac["account_value"] if if_using_sac else None,
        "mvo": MVO_result["Mean Var"],
    }
)

In [ ]:
result

,a2c,ddpg,ppo,td3,sac,mvo
date,,,,,,
2022-10-06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,9.938709e+05
2022-10-07,9.965399e+05,9.922338e+05,9.969623e+05,9.947430e+05,9.937270e+05,9.626415e+05
2022-10-10,9.939432e+05,9.881378e+05,9.955910e+05,9.914352e+05,9.913134e+05,9.582084e+05
2022-10-11,9.939316e+05,9.878620e+05,9.959989e+05,9.886916e+05,9.900869e+05,9.545301e+05
2022-10-12,9.930385e+05,9.877371e+05,9.953251e+05,9.875162e+05,9.890311e+05,9.525360e+05
...,...,...,...,...,...,...
2023-03-30,1.027562e+06,1.103167e+06,1.167983e+06,1.131246e+06,1.074472e+06,1.014359e+06
2023-03-31,1.041164e+06,1.117583e+06,1.184663e+06,1.146072e+06,1.086128e+06,1.025741e+06
2023-04-03,1.064739e+06,1.129697e+06,1.193259e+06,1.160568e+06,1.108352e+06,1.050658e+06


Now, everything is ready, we can plot the backtest result.

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()

<Axes: xlabel='date'>

In [ ]:
import numpy as np

def calculate_metrics(daily_value, risk_free_rate=0.03):
    # dropna
    daily_value = daily_value.dropna()
    final_value = daily_value.iloc[-1]
    initial_cash = daily_value.iloc[0]
    daily_returns = daily_value.pct_change().dropna()
    total_return = (final_value / initial_cash) - 1
    annual_return = (1 + total_return) ** (252 / len(daily_value)) - 1
    annual_volatility = daily_returns.std() * np.sqrt(252)
    downside_returns = daily_returns[daily_returns < 0]
    downside_deviation = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 1 else 0

    sharpe_ratio = (daily_returns.mean() * 252 - risk_free_rate) / annual_volatility if annual_volatility > 0 else 0
    sortino_ratio = (daily_returns.mean() * 252 - risk_free_rate) / downside_deviation if downside_deviation > 0 else 0

    return {
        "total_return": round(float(total_return), 3),
        "annual_return": round(float(annual_return), 3),
        "annual_volatility": round(float(annual_volatility), 3),
        "sharpe_ratio": round(float(sharpe_ratio), 3),
        "sortino_ratio": round(float(sortino_ratio), 3),
    }

methods = result.columns
metrics = pd.DataFrame()
for method in methods:
    metrics[method] = calculate_metrics(result[method])

metrics


,a2c,ddpg,ppo,td3,sac,mvo
total_return,0.072,0.132,0.186,0.157,0.122,0.061
annual_return,0.150,0.284,0.412,0.341,0.260,0.127
annual_volatility,0.176,0.175,0.216,0.175,0.164,0.221
sharpe_ratio,0.718,1.357,1.580,1.606,1.322,0.520
sortino_ratio,1.226,2.413,3.155,2.901,2.464,0.931
